This notebook generates the xon17 annotations, i.e. each gene is annotated as follows:
* 0 if the gene has no mutation
* 1 if the gene is mutated but has no deleterious mutations
* 1 + (x/17) if the mutation is deleterious according to `x` algorithms of Annovar. In reality first a sum over all 17 algorithms for each protein mutation is computed, followed by an average over the entire gene across all protein mutations.

In [ ]:
import sys

sys.path.append("../../vae_zinb_reprn/")
sys.path.append("../src/")

In [ ]:
import numpy as np
import pandas as pd

import datetime
import logging
import os
import time
from functools import cached_property

from sklearn.metrics import average_precision_score, ndcg_score, roc_auc_score

In [ ]:
from data import (
    construct_anno_features,
    construct_raw_mutation_features,
    construct_raw_cnv_features,
    canonicalize_mutations,
    ALIAS_TO_CANONICAL_NAME_MAP,
    construct_anno_features_xon17
)

genes_324 = pd.read_csv("../../data/druid_1.3_data/gene2ind.txt", header=None)[
    0
].tolist()

In [ ]:
%matplotlib inline

In [ ]:
# Set default logging level
# Change to logging.INFO to see related output
logging.basicConfig(level=logging.WARNING, force=True)
pd.set_option("display.max_columns", 20)

In [ ]:
df = pd.read_table(
    "../data/raw/CCLE.hg38.DepMap21Q3.annot.subset.cols.txt", low_memory=False,
)
print(df.shape)
df.head()

In [ ]:
ccle_mutations_df = pd.read_csv("../data/raw/CCLE_mutations.csv", low_memory=False,)
print(ccle_mutations_df.shape)
ccle_mutations_df.head()

In [ ]:
# ccle_cnv_df = pd.read_csv(
#     "../data/raw/CCLE_cn_segments_annotated_21Q3.txt", delimiter="\t"
# )
# ccle_cnv_df.rename(columns={"DepMap-ID": "depmap_id"}, inplace=True)
# print(ccle_cnv_df.shape)
# ccle_cnv_df.head()

In [ ]:
# ccle_cnv_pvt = pd.pivot_table(
#     ccle_cnv_df, "status", index=["depmap_id"], columns=["genesymbol"], aggfunc="first"
# )

# #
# # ccle_cnv_pvt = ccle_cnv_pvt.replace({'U':0, '+':1, '-':-1})

# # Binary
# ccle_cnv_pvt = ccle_cnv_pvt.replace({"U": 0, "+": 1, "-": 1})
# print(
#     "number of intersecting genes:",
#     np.intersect1d(ccle_cnv_pvt.columns.tolist(), genes_324).size,
# )

# missing_columns = np.setdiff1d(genes_324, ccle_cnv_pvt.columns.tolist())
# print(
#     "genes that are missing from depmap: ",
#     missing_columns,
# )

In [ ]:
# ccle_cnv_pvt.columns = [col.upper() for col in ccle_cnv_pvt.columns]

In [ ]:
# # mapping 'missing genes' to synonyms {ccle:324genes}
# # cnv_dict = {
# #     "GID4": "C17ORF39",
# #     "AMER1": "FAM123B",
# #     "KMT2A": "MLL",
# #     "KMT2D": "MLL2",
# #     "MYCL": "MYCL1",
# #     "MRE11": "MRE11A",
# #     "TENT5C": "FAM46C",
# #     "PRKN": "PARK2",
# #     "NSD2": "WHSC1",
# #     "NSD3": "WHSC1L1",
# # }
# # NOTE: use ALIAS_TO_CANONICAL_NAME_MAP instead of the above as it covers
# # lot more aliases than this list

# columns_to_rename = {
#     k: v for k, v in ALIAS_TO_CANONICAL_NAME_MAP.items() if v in missing_columns
# }
# ccle_cnv_pvt_renamed = ccle_cnv_pvt.rename(columns=columns_to_rename)

In [ ]:
# print(
#     "genes that are missing from depmap: ",
#     np.setdiff1d(genes_324, ccle_cnv_pvt_renamed.columns.tolist()),
# )

In [ ]:
# ccle_cnv_pvt_renamed["TERC"] = 0
# ccle_cnv_pvt_324 = ccle_cnv_pvt_renamed.loc[:, ccle_cnv_pvt_renamed.columns.isin(genes_324)].copy()

In [ ]:
# ccle_cnv_pvt_324.shape

In [ ]:
# for col in genes_324:
#     if len(ccle_cnv_pvt_324[col].shape) > 1:
#         print(col)

In [ ]:
# duplicated_columns = [
#     "BRIP1",
#     "CHEK2",
#     "CXCR4",
#     "DDR2",
#     "ERCC4",
#     "FANCA",
#     "FBXW7",
#     "FGFR1",
#     "GNAQ",
#     "HNF1A",
#     "IRF4",
#     "KDM5A",
#     "MED12",
#     "MEN1",
#     "NFE2L2",
#     "NKX2-1",
#     "PTEN",
#     "SOX9",
#     "XPO1",
# ]

# {k:v for k,v in ALIAS_TO_CANONICAL_NAME_MAP.items() if ((v in duplicated_columns)  and (k in ccle_cnv_pvt.columns))}

In [ ]:
# for col in ccle_cnv_pvt_324.columns:
#     ccle_cnv_pvt_324.loc[:, col] = ccle_cnv_pvt_324[col].astype(np.integer)

In [ ]:
# ccle_cnv_pvt_324

## Sample validation

In [ ]:
# ccle_cnv_df.genesymbol = ccle_cnv_df.genesymbol.replace(
#     list(columns_to_rename.keys()), list(columns_to_rename.values())
# )

In [ ]:
# unique_depmap_ids = ccle_cnv_pvt_324.index.get_level_values(0)
# rand_patient_idx = np.random.randint(0, len(unique_depmap_ids))
# depmap_id = unique_depmap_ids[rand_patient_idx]
# temp_df = ccle_cnv_df[ccle_cnv_df.depmap_id == depmap_id]
# display(temp_df[(temp_df.genesymbol.isin(genes_324)) & temp_df.status.isin(["+", "-"])])

# ccle_cnv_pvt_324.columns[ccle_cnv_pvt_324.loc[depmap_id].values != 0]

In [ ]:
# ccle_cnv_pvt_324.to_csv("../data/processed/ccle_cnv_binary.csv")

In [ ]:
df_auc = pd.read_csv("../../data/druid_1.3_data/cell_drug_auc_final_1111.csv")
df_auc["depmap_id"] = df_auc["ARXSPAN_ID"].astype("string")
df_auc.drop("ARXSPAN_ID", axis=1, inplace=True)
df_auc.set_index(["depmap_id"], inplace=True)
df_auc.head()

In [ ]:
ccle_mutations_df

In [ ]:
# mutations_needed = ["damaging", "other non-conserving"]

ccle_mutations_df = ccle_mutations_df[
    ["Hugo_Symbol", "DepMap_ID", "Protein_Change", "Variant_annotation"]
].copy()
print(ccle_mutations_df.shape)
ccle_mutations_filtered = ccle_mutations_df#[
#     ccle_mutations_df["Variant_annotation"].isin(mutations_needed)
# ].copy()
print(ccle_mutations_filtered.shape)
ccle_mutations_filtered.head()

In [ ]:
# Not filtering out any cell lines or mutations at this point

# ccle_mutations_filtered.rename(columns={"DepMap_ID": "depmap_id"}, inplace=True)
# ccle_mutations_filtered = ccle_mutations_filtered[
#     ccle_mutations_filtered.depmap_id.isin(df_auc.index.get_level_values(0))
# ]
# ccle_mutations_filtered

In [ ]:
ccle_mutations_filtered.Hugo_Symbol = ccle_mutations_filtered.Hugo_Symbol.str.upper()

In [ ]:
print(
    "number of intersecting genes:",
    np.intersect1d(ccle_mutations_filtered.Hugo_Symbol.unique(), genes_324).size,
)

missing_genes_in_mutations = np.setdiff1d(genes_324, ccle_mutations_filtered.Hugo_Symbol.unique())
print(
    "genes that are missing from depmap: ",
    missing_genes_in_mutations,
)

In [ ]:
# mut_dict = {
#     "GID4": "C17ORF39",
#     "C11orf30": "EMSY",
#     "AMER1": "FAM123B",
#     "KMT2A": "MLL",
#     "KMT2D": "MLL2",
#     "MYCL": "MYCL1",
# }
rows_to_rename_in_mutations = {
    k: v for k, v in ALIAS_TO_CANONICAL_NAME_MAP.items() if v in missing_genes_in_mutations
}
ccle_mutations_filtered.Hugo_Symbol.replace(
    list(rows_to_rename_in_mutations.keys()), list(rows_to_rename_in_mutations.values()), inplace=True
)

In [ ]:
print(
    "number of intersecting genes:",
    np.intersect1d(ccle_mutations_filtered.Hugo_Symbol.unique(), genes_324).size,
)

print(
    "genes that are missing from depmap: ",
    np.setdiff1d(genes_324, ccle_mutations_filtered.Hugo_Symbol.unique()),
)

In [ ]:
# Filtering out and using only the F1 genes
ccle_mutations_filtered = ccle_mutations_filtered[
    ccle_mutations_filtered.Hugo_Symbol.isin(genes_324)
]
# Not filtering out NaN protein mutations
# ccle_mutations_filtered = ccle_mutations_filtered[
#     ~ccle_mutations_filtered.Protein_Change.isna()
# ]
ccle_mutations_filtered["mutation"] = (
    ccle_mutations_filtered["Hugo_Symbol"]
    + " "
    + ccle_mutations_filtered["Protein_Change"].apply(lambda val: val[2:] if val is not np.NaN else "")
)
ccle_mutations_filtered["mutation"] = ccle_mutations_filtered["mutation"].str.strip(" ")
ccle_mutations_filtered

In [ ]:
# annotated_df = pd.DataFrame(columns=['input', 'sift_pred', 'sift4g_pred', 'lrt_pred', 'mutationtaster_pred',
#        'mutationassessor_pred', 'fathmm_pred', 'provean_pred', 'metasvm_pred',
#        'm_cap_pred', 'primateai_pred', 'deogen2_pred', 'bayesdel_addaf_pred',
#        'bayesdel_noaf_pred', 'clinpred_pred', 'list_s2_pred',
#        'fathmm_mkl_coding_pred', 'fathmm_xf_coding_pred', 'gene'])
# for depmap_id in ccle_mutations_filtered.depmap_id.unique():
#     filtered_df = ccle_mutations_filtered[
#         ccle_mutations_filtered.depmap_id == depmap_id
#     ]
#     point_mutations = list(filtered_df.mutation.values)
#     _, annot_df = construct_anno_features_xon17_gpd(depmap_id, point_mutations, False)
#     annotated_df = pd.concat([annotated_df, annot_df])

In [ ]:
# annotated_df.to_csv("../data/processed/anno_features_per_mutation_ccle.csv")

In [ ]:
# raw_mutations_combined = []
# for depmap_id in ccle_mutations_filtered.depmap_id.unique():
#     filtered_df = ccle_mutations_filtered[
#         ccle_mutations_filtered.depmap_id == depmap_id
#     ]
#     point_mutations = list(filtered_df.mutation.values)
#     raw_mutations_combined.append(
#         construct_raw_mutation_features(depmap_id, point_mutations)
#     )
# raw_mutations_combined_df = pd.concat(raw_mutations_combined)
# raw_mutations_combined_df.index.name = "depmap_id"
# raw_mutations_combined_df

In [ ]:
# rand_patient_idx = np.random.randint(0, 691)
# display(
#     ccle_mutations_filtered[
#         ccle_mutations_filtered.depmap_id
#         == raw_mutations_combined_df.iloc[rand_patient_idx].name
#     ]
# )

# raw_mutations_combined_df.columns[
#     raw_mutations_combined_df.iloc[rand_patient_idx].values != 0
# ]

In [ ]:
# raw_mutations_combined_df.to_csv("../data/processed/ccle_raw_mutation.csv")

## Variant Annotations with Transvar/Annovar

In [ ]:
# variant annotation on these point mutations
import sys
sys.path.append("../src/")
from data import (construct_anno_features, 
                  construct_raw_cnv_features, 
                  construct_raw_mutation_features, 
                  canonicalize_mutations,
                  ALIAS_TO_CANONICAL_NAME_MAP, 
                  _is_valid_point_mutations,
                  get_annotation_features,
                  preprocess_annotation_features,
                  MICEData
                 )

In [ ]:
import logging
import re
import subprocess
import tempfile
import itertools
import json

In [ ]:
REQUIRED_ANNOTATION_COLUMNS = [
    "SIFT_score",
    "SIFT_converted_rankscore",
    "SIFT_pred",
    "SIFT4G_score",
    "SIFT4G_converted_rankscore",
    "SIFT4G_pred",
    "LRT_score",
    "LRT_converted_rankscore",
    "LRT_pred",
    "MutationTaster_score",
    "MutationTaster_converted_rankscore",
    "MutationTaster_pred",
    "MutationAssessor_score",
    "MutationAssessor_rankscore",
    "MutationAssessor_pred",
    "FATHMM_score",
    "FATHMM_converted_rankscore",
    "FATHMM_pred",
    "PROVEAN_score",
    "PROVEAN_converted_rankscore",
    "PROVEAN_pred",
    "MetaSVM_pred",
    "M-CAP_score",
    "M-CAP_rankscore",
    "M-CAP_pred",
    "MVP_score",
    "MVP_rankscore",
    "MPC_score",
    "MPC_rankscore",
    "PrimateAI_score",
    "PrimateAI_rankscore",
    "PrimateAI_pred",
    "DEOGEN2_score",
    "DEOGEN2_rankscore",
    "DEOGEN2_pred",
    "BayesDel_addAF_score",
    "BayesDel_addAF_pred",
    "BayesDel_noAF_score",
    "BayesDel_noAF_rankscore",
    "BayesDel_noAF_pred",
    "ClinPred_score",
    "ClinPred_rankscore",
    "ClinPred_pred",
    "LIST-S2_score",
    "LIST-S2_rankscore",
    "LIST-S2_pred",
    "DANN_score",
    "DANN_rankscore",
    "fathmm-MKL_coding_score",
    "fathmm-MKL_coding_rankscore",
    "fathmm-MKL_coding_pred",
    "fathmm-XF_coding_score",
    "fathmm-XF_coding_rankscore",
    "fathmm-XF_coding_pred",
    "Eigen-raw_coding",
    "Eigen-raw_coding_rankscore",
    "Eigen-PC-raw_coding",
    "Eigen-PC-raw_coding_rankscore",
]
CATEGORICAL_COLUMNS = [
    "sift_pred",
    "sift4g_pred",
    "lrt_pred",
    "mutationtaster_pred",
    "mutationassessor_pred",
    "fathmm_pred",
    "provean_pred",
    "metasvm_pred",
    "m_cap_pred",
    "primateai_pred",
    "deogen2_pred",
    "bayesdel_addaf_pred",
    "bayesdel_noaf_pred",
    "clinpred_pred",
    "list_s2_pred",
    "fathmm_mkl_coding_pred",
    "fathmm_xf_coding_pred",
]

# The thresholds used in PREDICTOR_LAMBDA_MAP are taken from the corresponding
# technique's published paper/web page
PREDICTOR_LAMBDA_MAP = {
    "sift_pred": ("sift_score", lambda v: "D" if v <= 0.05 else "T"),
    "sift4g_pred": ("sift4g_score", lambda v: "D" if v <= 0.05 else "T"),
    "lrt_pred": ("lrt_score", lambda v: "D" if v <= 0.001 else "U"),
    "mutationtaster_pred": (
        "mutationtaster_score",
        lambda v: None,
    ),  # Threshold is not available and couldn't be derived from available values as well
    "mutationassessor_pred": (
        "mutationassessor_score",
        lambda v: "H" if v >= 3.5 else ("M" if v >= 1.94 else "L"),
    ),
    "fathmm_pred": ("fathmm_score", lambda v: "D" if v < 1.5 else "T"),
    "provean_pred": ("provean_score", lambda v: "D" if v <= 2.282 else "N"),
    "metasvm_pred": (
        "metasvm_pred",
        lambda v: None,
    ),  # No corresponding numeric score available for this method
    "m_cap_pred": ("m_cap_score", lambda v: "D" if v >= 0.025 else "T"),
    "primateai_pred": ("primateai_score", lambda v: "D" if v >= 0.803 else "T"),
    "deogen2_pred": ("deogen2_score", lambda v: "D" if v >= 0.45 else "T"),
    "bayesdel_addaf_pred": (
        "bayesdel_addaf_score",
        lambda v: "D" if v >= 0.0692 else "T",
    ),
    "bayesdel_noaf_pred": (
        "bayesdel_noaf_score",
        lambda v: "D" if v >= -0.0570 else "T",
    ),
    "clinpred_pred": ("clinpred_score", lambda v: "D" if v >= 0.5 else "T"),
    "list_s2_pred": ("list_s2_score", lambda v: "D" if v >= 0.85 else "T"),
    "fathmm_mkl_coding_pred": (
        "fathmm_mkl_coding_score",
        lambda v: "D" if v >= 0.5 else "N",
    ),
    "fathmm_xf_coding_pred": (
        "fathmm_xf_coding_score",
        lambda v: "D" if v >= 0.5 else "N",
    ),
}

DELETERIOUS_VALUES = ["D", "A", "H", "M"]

CNV_PATTERN = r"loss|amplification"

GENES_324 = pd.read_csv("../../data/druid_1.3_data/gene2ind.txt", header=None)[
    0
].tolist()

ANNOTATION_SCRIPT_PATH = "../script/goAAtoGv2.sh"
SPECIAL_CASES = r"rearrangement|truncation|fs|del|ins"

In [ ]:
# variant annotation on these point mutations
import sys
sys.path.append("../src/")
from data import (construct_anno_features, 
                  construct_raw_cnv_features, 
                  construct_raw_mutation_features, 
                  canonicalize_mutations,
                  ALIAS_TO_CANONICAL_NAME_MAP, 
                  _is_valid_point_mutations,
                  get_annotation_features,
                  preprocess_annotation_features,
                  MICEData
                 )

import logging
import re
import subprocess
import tempfile
import itertools
import json

REQUIRED_ANNOTATION_COLUMNS = [
    "SIFT_score",
    "SIFT_converted_rankscore",
    "SIFT_pred",
    "SIFT4G_score",
    "SIFT4G_converted_rankscore",
    "SIFT4G_pred",
    "LRT_score",
    "LRT_converted_rankscore",
    "LRT_pred",
    "MutationTaster_score",
    "MutationTaster_converted_rankscore",
    "MutationTaster_pred",
    "MutationAssessor_score",
    "MutationAssessor_rankscore",
    "MutationAssessor_pred",
    "FATHMM_score",
    "FATHMM_converted_rankscore",
    "FATHMM_pred",
    "PROVEAN_score",
    "PROVEAN_converted_rankscore",
    "PROVEAN_pred",
    "MetaSVM_pred",
    "M-CAP_score",
    "M-CAP_rankscore",
    "M-CAP_pred",
    "MVP_score",
    "MVP_rankscore",
    "MPC_score",
    "MPC_rankscore",
    "PrimateAI_score",
    "PrimateAI_rankscore",
    "PrimateAI_pred",
    "DEOGEN2_score",
    "DEOGEN2_rankscore",
    "DEOGEN2_pred",
    "BayesDel_addAF_score",
    "BayesDel_addAF_pred",
    "BayesDel_noAF_score",
    "BayesDel_noAF_rankscore",
    "BayesDel_noAF_pred",
    "ClinPred_score",
    "ClinPred_rankscore",
    "ClinPred_pred",
    "LIST-S2_score",
    "LIST-S2_rankscore",
    "LIST-S2_pred",
    "DANN_score",
    "DANN_rankscore",
    "fathmm-MKL_coding_score",
    "fathmm-MKL_coding_rankscore",
    "fathmm-MKL_coding_pred",
    "fathmm-XF_coding_score",
    "fathmm-XF_coding_rankscore",
    "fathmm-XF_coding_pred",
    "Eigen-raw_coding",
    "Eigen-raw_coding_rankscore",
    "Eigen-PC-raw_coding",
    "Eigen-PC-raw_coding_rankscore",
]
CATEGORICAL_COLUMNS = [
    "sift_pred",
    "sift4g_pred",
    "lrt_pred",
    "mutationtaster_pred",
    "mutationassessor_pred",
    "fathmm_pred",
    "provean_pred",
    "metasvm_pred",
    "m_cap_pred",
    "primateai_pred",
    "deogen2_pred",
    "bayesdel_addaf_pred",
    "bayesdel_noaf_pred",
    "clinpred_pred",
    "list_s2_pred",
    "fathmm_mkl_coding_pred",
    "fathmm_xf_coding_pred",
]

# The thresholds used in PREDICTOR_LAMBDA_MAP are taken from the corresponding
# technique's published paper/web page
PREDICTOR_LAMBDA_MAP = {
    "sift_pred": ("sift_score", lambda v: "D" if v <= 0.05 else "T"),
    "sift4g_pred": ("sift4g_score", lambda v: "D" if v <= 0.05 else "T"),
    "lrt_pred": ("lrt_score", lambda v: "D" if v <= 0.001 else "U"),
    "mutationtaster_pred": (
        "mutationtaster_score",
        lambda v: None,
    ),  # Threshold is not available and couldn't be derived from available values as well
    "mutationassessor_pred": (
        "mutationassessor_score",
        lambda v: "H" if v >= 3.5 else ("M" if v >= 1.94 else "L"),
    ),
    "fathmm_pred": ("fathmm_score", lambda v: "D" if v < 1.5 else "T"),
    "provean_pred": ("provean_score", lambda v: "D" if v <= 2.282 else "N"),
    "metasvm_pred": (
        "metasvm_pred",
        lambda v: None,
    ),  # No corresponding numeric score available for this method
    "m_cap_pred": ("m_cap_score", lambda v: "D" if v >= 0.025 else "T"),
    "primateai_pred": ("primateai_score", lambda v: "D" if v >= 0.803 else "T"),
    "deogen2_pred": ("deogen2_score", lambda v: "D" if v >= 0.45 else "T"),
    "bayesdel_addaf_pred": (
        "bayesdel_addaf_score",
        lambda v: "D" if v >= 0.0692 else "T",
    ),
    "bayesdel_noaf_pred": (
        "bayesdel_noaf_score",
        lambda v: "D" if v >= -0.0570 else "T",
    ),
    "clinpred_pred": ("clinpred_score", lambda v: "D" if v >= 0.5 else "T"),
    "list_s2_pred": ("list_s2_score", lambda v: "D" if v >= 0.85 else "T"),
    "fathmm_mkl_coding_pred": (
        "fathmm_mkl_coding_score",
        lambda v: "D" if v >= 0.5 else "N",
    ),
    "fathmm_xf_coding_pred": (
        "fathmm_xf_coding_score",
        lambda v: "D" if v >= 0.5 else "N",
    ),
}

DELETERIOUS_VALUES = ["D", "A", "H", "M"]

CNV_PATTERN = r"loss|amplification"

GENES_324 = pd.read_csv("../../data/druid_1.3_data/gene2ind.txt", header=None)[
    0
].tolist()

ANNOTATION_SCRIPT_PATH = "../script/goAAtoGv2.sh"
SPECIAL_CASES = r"rearrangement|truncation|fs|del|ins"

In [ ]:
def construct_anno_features_xon17_gpd(patient_id, patient_mutations, agg_features=False):
    """
    TODO: Add support for other agg functions (mean, OR, etc) - as of 202209, only
    sum is supported
    Here, the aggregation is done as an average over all variants over all 17 algorithms.
    """
    if agg_features:
        logging.warn(
            """
        Received agg_features=True -> As of now, construct_anno_features only supports sum aggregation.
        Please ensure that the agg used in dataset definition is sum - if it is not sum, please pass
        agg_features=False and perform agg in dataset definition
        """
        )

#     if not _is_valid_point_mutations(patient_mutations):
#         return None

    anno_features_combined_imputed_df = pd.read_csv(
        "../data/processed/anno_features_combined_imputed.csv"
    )
    logging.info(anno_features_combined_imputed_df.shape)
    anno_features_combined_imputed_df.set_index(["input"], inplace=True)
    anno_features_combined_imputed_df.head()

    canonical_mutations = canonicalize_mutations(patient_mutations)

    mutations_with_missing_annotations = []
    available_mutations = []
    for mutation in canonical_mutations:
        if mutation in anno_features_combined_imputed_df.index:
            available_mutations.append(mutation)
        elif not re.search(CNV_PATTERN, mutation, re.IGNORECASE):
            mutations_with_missing_annotations.append(mutation)

    if available_mutations:
        patient_anno_features = anno_features_combined_imputed_df.loc[
            available_mutations
        ]
        patient_anno_features = patient_anno_features[CATEGORICAL_COLUMNS].copy()
    else:
        patient_anno_features = None

    if len(mutations_with_missing_annotations) != 0:
        logging.info(
            f"Found mutations with missing annotations - {mutations_with_missing_annotations}"
        )
        missing_annotations = get_annotation_features(
            mutations_with_missing_annotations
        )
        if missing_annotations is not None:
            missing_annotations = missing_annotations[
                REQUIRED_ANNOTATION_COLUMNS
            ].copy()
            missing_annotations.reset_index(inplace=True)
            missing_annotations = preprocess_annotation_features(missing_annotations)
            missing_annotations = missing_annotations[~missing_annotations.duplicated()]
            missing_annotations.reset_index(drop=True, inplace=True)
            missing_annotations.set_index("input", inplace=True)

            numeric_columns = list(
                column
                for column in missing_annotations.columns
                if pd.api.types.is_numeric_dtype(missing_annotations[column])
            )
            # Prepare mask by identifying rows that have all na values for numeric_columns
            na_mask = None
            for col in numeric_columns:
                if type(na_mask) == pd.Series:
                    na_mask = na_mask & missing_annotations[col].isna()
                else:
                    na_mask = missing_annotations[col].isna()

            missing_annotations = missing_annotations[~na_mask]
            numeric_df = missing_annotations[numeric_columns].copy()
            logging.info(numeric_df.shape)
            numeric_df.head()
            numeric_df = pd.concat(
                [numeric_df, anno_features_combined_imputed_df[numeric_columns]],
            )

            categorical_columns = [
                column
                for column in missing_annotations.columns
                if column not in numeric_columns
            ]
            categorical_missing_annotations = missing_annotations[
                categorical_columns
            ].copy()
            logging.info(categorical_missing_annotations.shape)
            categorical_missing_annotations.head()

            imp = MICEData(numeric_df)
            # Impute missing values in numeric columns - Expensive!!
            imp.update_all()
            imputed_df = imp.data
            assert numeric_df.shape == imputed_df.shape
            imputed_df.index = numeric_df.index
            imputed_df = imputed_df[
                imputed_df.index.isin(mutations_with_missing_annotations)
            ].copy()
            numeric_imputed_df = pd.concat(
                [categorical_missing_annotations, imputed_df,], axis=1,
            )
            logging.info(numeric_imputed_df.shape)
            for column in CATEGORICAL_COLUMNS:
                logging.info(
                    column,
                    numeric_imputed_df[column].unique(),
                    len(numeric_imputed_df[numeric_imputed_df[column].isna()]),
                )
                col_na_mask = numeric_imputed_df[column].isna()
                numeric_imputed_df.loc[col_na_mask, column] = numeric_imputed_df[
                    col_na_mask
                ][PREDICTOR_LAMBDA_MAP[column][0]].apply(
                    PREDICTOR_LAMBDA_MAP[column][1]
                )

                # logging.info(
                #     column,
                #     numeric_imputed_df[column].unique(),
                #     len(numeric_imputed_df[numeric_imputed_df[column].isna()]),
                # )

            numeric_imputed_df = numeric_imputed_df.dropna()
            logging.info(numeric_imputed_df.shape)
            missing_anno_features_df = numeric_imputed_df[CATEGORICAL_COLUMNS].copy()

            patient_anno_features = pd.concat(
                [missing_anno_features_df, patient_anno_features]
            )
            
    if patient_anno_features is not None:
        for col in patient_anno_features.columns:
            patient_anno_features[col] = patient_anno_features[col].apply(
                lambda v: 1 if v in DELETERIOUS_VALUES else 0
            )

        patient_anno_features.reset_index(inplace=True)
        patient_anno_features["gene"] = patient_anno_features.input.apply(
            lambda gene_mut: gene_mut.split(" ")[0]
        )
    return patient_anno_features

In [ ]:
# annotated_df = pd.read_csv("../data/processed/anno_features_per_mutation_ccle.csv", index_col = 0)
# annotated_df

In [ ]:
# len(set(ccle_mutations_filtered["mutation"]) - set(annotated_df["input"]))

In [ ]:
# set(ccle_mutations_filtered["mutation"]) - set(annotated_df["input"])

In [ ]:
import dask

from tqdm import tqdm
from dask.distributed import Client


client = Client()
client.cluster.scale(10)

In [ ]:
%load_ext jupyterlab_notify

In [ ]:
%%notify

# futures = []
anno_features_combined = pd.DataFrame()
for depmap_id in ccle_mutations_filtered.DepMap_ID.unique():
    filtered_df = ccle_mutations_filtered[
        ccle_mutations_filtered.DepMap_ID == depmap_id
    ]
    point_mutations = list(filtered_df.mutation.values)
    res = construct_anno_features_xon17_gpd(depmap_id, point_mutations, False)
    anno_features_combined = pd.concat([anno_features_combined, res], ignore_index = False)
#     future = client.submit(construct_anno_features_xon17_gpd, depmap_id, point_mutations, False)
#     futures.append(future)

# anno_features_combined = client.gather(futures, errors="skip")
# client.shutdown()

In [ ]:
anno_features_combined

In [ ]:
# filtered_df = ccle_mutations_filtered[
#     ccle_mutations_filtered.depmap_id == "ACH-001716"
# ]
# point_mutations = list(filtered_df.mutation.values)
# test = construct_anno_features_xon17("ACH-001716", point_mutations, False)

In [ ]:
# point_mutations

In [ ]:
# test

In [ ]:
agg_anno_features_combined_df = anno_features_combined#pd.concat(anno_features_combined).reset_index()
# agg_anno_features_combined_df.rename(columns={"patient_id": "depmap_id"}, inplace=True)
agg_anno_features_combined_df.set_index("input", inplace=True)
agg_anno_features_combined_df

In [ ]:
ccle_mutations_filtered

In [ ]:
# one more round of annotation to confirm
append_df = pd.DataFrame()
for mut in set(ccle_mutations_filtered.mutation) - set(agg_anno_features_combined_df.index):
    res_df = construct_anno_features_xon17_gpd("dummy_patient", [mut], False)
    append_df = pd.concat([append_df, res_df], ignore_index=True)

In [ ]:
append_df # Run interrupted to check if there is any update happening

In [ ]:
len(set(ccle_mutations_filtered.mutation) - (set(agg_anno_features_combined_df.index)) | set(append_df.input))

In [ ]:
# whatever is left ~ 22k mutations

remaining_df = pd.DataFrame()
for mut in set(ccle_mutations_filtered.mutation) - (set(agg_anno_features_combined_df.index) | set(append_df.input)):
    res_df = construct_anno_features_xon17_gpd("dummy_patient", [mut], False)
    remaining_df = pd.concat([remaining_df, res_df], ignore_index=True)

In [ ]:
remaining_df

In [ ]:
## HERE!!!

In [ ]:
# mutations without annotations
len(set(ccle_mutations_filtered.mutation) - (set(agg_anno_features_combined_df.index) | set(append_df.input) | set(remaining_df.input)))

In [ ]:
len(set(pd.read_csv("../data/processed/anno_features_per_mutation_ccle.csv")["input"].unique())), \
len(set(agg_anno_features_combined_df.index) | set(append_df.input) | set(remaining_df.input))

In [ ]:
set(pd.read_csv("../data/processed/anno_features_per_mutation_ccle.csv")["input"].unique()) - \
(set(agg_anno_features_combined_df.index) | set(append_df.input) | set(remaining_df.input))

In [ ]:
# Overwriting existing anno_features_per_mutation_ccle.csv file
concatenated_df = pd.concat([agg_anno_features_combined_df.reset_index(), append_df, remaining_df],ignore_index=True)
concatenated_df

In [ ]:
len(set(concatenated_df["input"]))

In [ ]:
len(set(ccle_mutations_filtered["mutation"])) # There are ~ 12k mutations that are missing annotations

In [ ]:
concatenated_df.set_index("input", inplace=True, drop=True)

In [ ]:
# concatenated_df.to_csv("../data/processed/anno_features_per_mutation_ccle.csv")

In [ ]:
concatenated_df = pd.read_csv("../data/processed/anno_features_per_mutation_ccle.csv", index_col = 0)
concatenated_df

In [ ]:
ccle_mutations_filtered

In [ ]:
len(ccle_mutations_filtered["DepMap_ID"].unique()), ccle_mutations_filtered.shape

In [ ]:
merged_1 = pd.merge(ccle_mutations_filtered, concatenated_df.reset_index(), left_on = "mutation", right_on="input", how="left")
merged_1

In [ ]:
merged_1.drop("input", axis = 1, inplace=True)
merged_1.isna().sum()

In [ ]:
# For those point mutations without annotations
merged_1.fillna(0, inplace=True)

In [ ]:
merged_1

In [ ]:
merged_1["HGVSp"] = merged_1["mutation"].apply(lambda x: "p." + x.split(" ")[1] if len(x.split(" ")) >= 2 else "p.")

In [ ]:
len(merged_1["HGVSp"].unique())

In [ ]:
len(merged_1["mutation"].unique()) # this includes gene name as well

### ClinVar annotations

In [ ]:
import tempfile
import logging
import subprocess
ANNOTATION_SCRIPT_PATH_CLINVAR = "../script/goAAtoGv2_clinvar.sh"

In [ ]:
results = []
for mutation in list(merged_1["mutation"]):
    try:
        # Run annotation script within a temp file and extract features as DataFrame
        with tempfile.TemporaryDirectory() as tmpdirname:
            input_file_path = tmpdirname + "anno_input.txt"
            with open(input_file_path, "w+") as input_file:
                mutation_cleaned = [part for part in mutation.split(" ") if part]
                input_file.write(":p.".join(mutation_cleaned))
                input_file.write("\n")

            # Execute script
            cmd = "bash {0} {1}".format(ANNOTATION_SCRIPT_PATH_CLINVAR, input_file_path)

            logging.info(f"Executing command {cmd}")
            subprocess.call(cmd, shell=True, executable="/bin/bash")
            out_file_path = f"{input_file_path}.annot.hg38_finalannot.txt"
            res = pd.read_table(out_file_path)
    # Some inputs lead to errors, such as "PTEN loss" - ignore and continue processing
    except Exception as e:
        logging.error(
            f"Encountered error while processing mutation {mutation} - {e}"
        )
        continue
    res["input"] = mutation
    res = res[~res.duplicated()]
    results.append(res)
clinvar_annot_df = pd.concat(results)
clinvar_annot_df.set_index(["input"], inplace=True)
clinvar_annot_df.drop(columns=["Otherinfo1"], inplace=True)

In [ ]:
len(results)

In [ ]:
clinvar_annot_df = pd.concat(results)
clinvar_annot_df.set_index(["input"], inplace=True)
clinvar_annot_df.drop(columns=["Otherinfo1"], inplace=True)

In [ ]:
clinvar_annot_df

In [ ]:
clinvar_annot_df.CLNSIG.value_counts()

In [ ]:
remaining_mut = list(set(merged_1["mutation"]) - set(clinvar_annot_df.index))
len(set(merged_1["mutation"]) - set(clinvar_annot_df.index))

In [ ]:
# one more round with all the mutations that were missed out before the earlier interrupt
results2 = []
for mutation in remaining_mut:
    try:
        # Run annotation script within a temp file and extract features as DataFrame
        with tempfile.TemporaryDirectory() as tmpdirname:
            input_file_path = tmpdirname + "anno_input.txt"
            with open(input_file_path, "w+") as input_file:
                mutation_cleaned = [part for part in mutation.split(" ") if part]
                input_file.write(":p.".join(mutation_cleaned))
                input_file.write("\n")

            # Execute script
            cmd = "bash {0} {1}".format(ANNOTATION_SCRIPT_PATH_CLINVAR, input_file_path)

            logging.info(f"Executing command {cmd}")
            subprocess.call(cmd, shell=True, executable="/bin/bash")
            out_file_path = f"{input_file_path}.annot.hg38_finalannot.txt"
            res = pd.read_table(out_file_path)
    # Some inputs lead to errors, such as "PTEN loss" - ignore and continue processing
    except Exception as e:
        logging.error(
            f"Encountered error while processing mutation {mutation} - {e}"
        )
        continue
    res["input"] = mutation
    res = res[~res.duplicated()]
    results2.append(res)

In [ ]:
clinvar_annot_df2 = pd.concat(results2)
clinvar_annot_df2.set_index(["input"], inplace=True)
clinvar_annot_df2.drop(columns=["Otherinfo1"], inplace=True)
clinvar_annot_df2

In [ ]:
clinvar_annot_df_overall = pd.concat([clinvar_annot_df, clinvar_annot_df2], axis = 0)
clinvar_annot_df_overall

In [ ]:
clinvar_annot_df_overall.CLNSIG.value_counts()

In [ ]:
clinvar_annot_df_overall.to_csv("../data/processed/clinvar_anno_features_per_mutation_ccle.csv")

### Output: Revised Annovar Xon17 score

In [ ]:
merged_1["1plusxon17_score"] = 1 + merged_1[CATEGORICAL_COLUMNS].sum(axis=1)/17
merged_1

In [ ]:
patient_gene_matrix_xon17 = merged_1.pivot_table(index="DepMap_ID", columns="Hugo_Symbol", values="1plusxon17_score", aggfunc="max")
patient_gene_matrix_xon17.fillna(0, inplace=True)
patient_gene_matrix_xon17

In [ ]:
for g in GENES_324:
    if g not in patient_gene_matrix_xon17.columns:
        patient_gene_matrix_xon17[g] = 0
patient_gene_matrix_xon17.shape

In [ ]:
# patient_gene_matrix_xon17[GENES_324].reset_index().rename(columns={"DepMap_ID": "depmap_id"}).to_csv("../data/processed/ccle_anno_features_xon17.csv", index=False)

### Variant Annotations from GPD

In [ ]:
# Load intermediate GPD files for NPC and PC

In [ ]:
npc_mutations = pd.read_csv("../data/processed/ccle_21q3_gpd_results/ccle_21q3_mutation_npc.tsv", sep="\t")
npc_mutations

In [ ]:
npc_mutations["HGVSp"].value_counts(dropna=False)

In [ ]:
npc_mutations["Variant_Classification"].value_counts(dropna=False)

In [ ]:
pc_mutations = pd.read_csv("../data/processed/ccle_21q3_gpd_results/ccle_21q3_mutation_pc_pos.tsv", sep="\t")
pc_mutations

In [ ]:
pc_mutations["Variant_Classification"].value_counts(dropna=False)

In [ ]:
# To get PIU vs LU, we used the locations in ptm_pfam_combine.csv which is used in the GPD implementation
ptm_pfam_df = pd.read_csv("/data/ajayago/druid/datasets/ptm_pfam_combine.csv", index_col = 0)
ptm_pfam_df

In [ ]:
GPD_unit = []
for idx, row in pc_mutations.iterrows():
    subset_ptm = ptm_pfam_df[ptm_pfam_df.gene_id == row["Gene"]]
    x = "LU"
    for idx, r in subset_ptm.iterrows():
        if (row["prot_start_pos"] >= r["start_position"]) & (row["prot_start_pos"] <= r["end_position"]) \
    | (row["prot_end_pos"] >= r["start_position"]) & (row["prot_end_pos"] <= r["end_position"]):
            x = "PIU"
            break
    GPD_unit.append(x)

pc_mutations["GPD_unit"] = GPD_unit

In [ ]:
pc_mutations["GPD_unit"].value_counts()

In [ ]:
piu_mutations = set(pc_mutations[pc_mutations["GPD_unit"] == "PIU"]["Hugo_Symbol"] + " " + pc_mutations[pc_mutations["GPD_unit"] == "PIU"]["HGVSp"].apply(lambda x: x.split("p.")[1]))
lu_mutations = set(pc_mutations[pc_mutations["GPD_unit"] == "LU"]["Hugo_Symbol"] + " " + pc_mutations[pc_mutations["GPD_unit"] == "LU"]["HGVSp"].apply(lambda x: x.split("p.")[1]))
len(piu_mutations), len(lu_mutations)

In [ ]:
# Map each point mutation to PIU/LU or NCU
GPD_unit_merged1 = []
for idx, row in merged_1.iterrows():
    if row["mutation"] in (piu_mutations):
        GPD_unit_merged1.append("PIU")
    elif row["mutation"] in (lu_mutations):
        GPD_unit_merged1.append("LU")
    else:
        GPD_unit_merged1.append("NCU")

In [ ]:
merged_1["GPD_unit"] = GPD_unit_merged1

In [ ]:
merged_1

### Output: GPD based Annovar annotation

In [ ]:
patient_gene_matrix_xon17_piu = merged_1[merged_1.GPD_unit == "PIU"].pivot_table(index="DepMap_ID", columns="Hugo_Symbol", values="1plusxon17_score", aggfunc="max")
patient_gene_matrix_xon17_piu.fillna(0, inplace=True)
for g in GENES_324:
    if g not in patient_gene_matrix_xon17_piu.columns:
        patient_gene_matrix_xon17_piu[g] = 0

patient_gene_matrix_xon17_piu

In [ ]:
patient_gene_matrix_xon17_lu = merged_1[merged_1.GPD_unit == "LU"].pivot_table(index="DepMap_ID", columns="Hugo_Symbol", values="1plusxon17_score", aggfunc="max")
patient_gene_matrix_xon17_lu.fillna(0, inplace=True)
for g in GENES_324:
    if g not in patient_gene_matrix_xon17_lu.columns:
        patient_gene_matrix_xon17_lu[g] = 0
patient_gene_matrix_xon17_lu

In [ ]:
patient_gene_matrix_xon17_ncu = merged_1[merged_1.GPD_unit == "NCU"].pivot_table(index="DepMap_ID", columns="Hugo_Symbol", values="1plusxon17_score", aggfunc="max")
patient_gene_matrix_xon17_ncu.fillna(0, inplace=True)
for g in GENES_324:
    if g not in patient_gene_matrix_xon17_ncu.columns:
        patient_gene_matrix_xon17_ncu[g] = 0
patient_gene_matrix_xon17_ncu

In [ ]:
patient_gene_matrix_xon17_piu.shape, patient_gene_matrix_xon17_lu.shape, patient_gene_matrix_xon17_ncu.shape

In [ ]:
len(set(patient_gene_matrix_xon17.index) - set(patient_gene_matrix_xon17_piu.index))

In [ ]:
# Add in missing patient IDs in each matrix
patient_gene_matrix_xon17_piu.reset_index(inplace=True)
patient_gene_matrix_xon17_lu.reset_index(inplace=True)
patient_gene_matrix_xon17_ncu.reset_index(inplace=True)
for t in set(patient_gene_matrix_xon17.index) - set(patient_gene_matrix_xon17_piu.DepMap_ID):
    patient_gene_matrix_xon17_piu = (patient_gene_matrix_xon17_piu.append({"DepMap_ID": t}, ignore_index=True))
patient_gene_matrix_xon17_piu.set_index("DepMap_ID", inplace=True)    
for t in set(patient_gene_matrix_xon17.index) - set(patient_gene_matrix_xon17_lu.DepMap_ID):
    patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
patient_gene_matrix_xon17_lu.set_index("DepMap_ID", inplace=True)
for t in set(patient_gene_matrix_xon17.index) - set(patient_gene_matrix_xon17_ncu.DepMap_ID):
    patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
patient_gene_matrix_xon17_ncu.set_index("DepMap_ID", inplace=True)  

In [ ]:
patient_gene_matrix_xon17_piu.shape, patient_gene_matrix_xon17_lu.shape, patient_gene_matrix_xon17_ncu.shape

In [ ]:
patient_gene_matrix_xon17_piu.drop("index", axis =1, inplace=True)
patient_gene_matrix_xon17_lu.drop("index", axis =1, inplace=True)
patient_gene_matrix_xon17_ncu.drop("index", axis =1, inplace=True)

In [ ]:
patient_gene_matrix_xon17_piu.fillna(0, inplace=True)
patient_gene_matrix_xon17_lu.fillna(0, inplace=True)
patient_gene_matrix_xon17_ncu.fillna(0, inplace=True)

In [ ]:
patient_gene_matrix_xon17_piu.loc[patient_gene_matrix_xon17.index][GENES_324].to_csv("../data/processed/xon17_gpd_annotations/ccle_21q3_piu_annotated_df.csv")

In [ ]:
patient_gene_matrix_xon17_lu.loc[patient_gene_matrix_xon17.index][GENES_324].to_csv("../data/processed/xon17_gpd_annotations/ccle_21q3_lu_annotated_df.csv")

In [ ]:
patient_gene_matrix_xon17_ncu.loc[patient_gene_matrix_xon17.index][GENES_324].to_csv("../data/processed/xon17_gpd_annotations/ccle_21q3_ncu_annotated_df.csv")

In [ ]:
pd.read_csv("../data/processed/xon17_gpd_annotations/ccle_21q3_piu_annotated_df.csv").set_index("DepMap_ID")

In [ ]:
merged_1.to_csv("../data/processed/ccle_21q3_annovar_gpd_annot_per_patient_per_mutation.csv", index=False)

#### Combine Clinvar annotation, Annovar and GPD annotations

In [ ]:
merged_clinvar_df = pd.merge(merged_1, clinvar_annot_df_overall.reset_index(), left_on="mutation", right_on="input", how="outer").groupby("mutation").aggregate(max)

In [ ]:
merged_clinvar_df[(merged_clinvar_df.CLNSIG != ".")&(~merged_clinvar_df.CLNSIG.isna())&(merged_clinvar_df.CLNSIG=="Pathogenic")][["GPD_unit", "CLNSIG", ]]

In [ ]:
merged_clinvar_df.groupby(["GPD_unit", "CLNSIG"]).aggregate("count")

In [ ]:
merged_clinvar_df.columns

In [ ]:
merged_1[merged_1.mutation == "ABL1 G259G"]

In [ ]:
merged_1["1plusxon17_score"] = 1 + merged_1[CATEGORICAL_COLUMNS].sum(axis=1)/17
merged_1

In [ ]:
merged_1[merged_1.mutation == "AKT1 E17K"]